## TC 5033
### Word Embeddings

<br>

#### Activity 3b: Text Classification using RNNs and AG_NEWS dataset in PyTorch
<br>

- Objective:
    - Understand the basics of Recurrent Neural Networks (RNNs) and their application in text classification.
    - Learn how to handle a real-world text dataset, AG_NEWS, in PyTorch.
    - Gain hands-on experience in defining, training, and evaluating a text classification model in PyTorch.
    
<br>

- Instructions:
    - Data Preparation: Starter code will be provided that loads the AG_NEWS dataset and prepares it for training. Do not modify this part. However, you should be sure to understand it, and comment it, the use of markdown cells is suggested.

    - Model Setup: A skeleton code for the RNN model class will be provided. Complete this class and use it to instantiate your model.

    - Implementing Accuracy Function: Write a function that takes model predictions and ground truth labels as input and returns the model's accuracy.

    - Training Function: Implement a function that performs training on the given model using the AG_NEWS dataset. Your model should achieve an accuracy of at least 80% to get full marks for this part.

    - Text Sampling: Write a function that takes a sample text as input and classifies it using your trained model.

    - Confusion Matrix: Implement a function to display the confusion matrix for your model on the test data.

    - Submission: Submit your completed Jupyter Notebook. Make sure to include a markdown cell at the beginning of the notebook that lists the names of all team members. Teams should consist of 3 to 4 members.
    
<br>

- Evaluation Criteria:

    - Correct setup of all the required libraries and modules (10%)
    - Code Quality (30%): Your code should be well-organized, clearly commented, and easy to follow. Use also markdown cells for clarity. Comments should be given for all the provided code, this will help you understand its functionality.
    
   - Functionality (60%):
        - All the functions should execute without errors and provide the expected outputs.
        - RNN model class (20%)
        - Accuracy fucntion (10%)
        - Training function (10%)
        - Sampling function (10%)
        - Confucion matrix (10%)

        - The model should achieve at least an 80% accuracy on the AG_NEWS test set for full marks in this criterion.


Dataset

https://pytorch.org/text/stable/datasets.html#text-classification

https://paperswithcode.com/dataset/ag-news


#### Install libraries (if needed)

## Import libraries

In [24]:
# The following libraries are required for running the given code
# Please feel free to add any libraries you consider adecuate to complete the assingment.
import numpy as np
#PyTorch libraries
import torch
from torchtext.datasets import AG_NEWS
# Dataloader library
from torch.utils.data import DataLoader
from torch.utils.data.dataset import random_split
# Libraries to prepare the data
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torchtext.data.functional import to_map_style_dataset
# neural layers
from torch import nn
from torch.nn import functional as F

# These libraries are suggested to plot confusion matrix
# you may use others
import scikitplot as skplt
import gc

In [25]:
# Use GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


### Get the train and the test datasets and dataloaders

Classes:

* 1 - World

* 2 - Sports

* 3 - Business

* 4 - Sci/Tech

We will convert them to:

* 0 - World

* 1 - Sports

* 2 - Business

* 3 - Sci/Tech

In [26]:
train_dataset, test_dataset = AG_NEWS()
train_dataset, test_dataset = to_map_style_dataset(train_dataset), to_map_style_dataset(test_dataset)

In [27]:
# Get the tokeniser
# tokeniser object
tokeniser = get_tokenizer('basic_english')

def yield_tokens(data):
    for _, text in data:
        yield tokeniser(text)

In [28]:
# Build the vocabulary
vocab = build_vocab_from_iterator(yield_tokens(train_dataset), specials=["<unk>"])
#set unknown token at position 0
vocab.set_default_index(vocab["<unk>"])

In [29]:
#test tokens
tokens = tokeniser('Welcome to TC5033')
print(tokens, vocab(tokens))

['welcome', 'to', 'tc5033'] [3314, 4, 0]


In [30]:
NUM_TRAIN = int(len(train_dataset)*0.9)
NUM_VAL = len(train_dataset) - NUM_TRAIN

In [31]:
train_dataset, val_dataset = random_split(train_dataset, [NUM_TRAIN, NUM_VAL])

In [32]:
print(len(train_dataset), len(val_dataset), len(test_dataset))

108000 12000 7600


In [33]:
labels =  ["World", "Sports", "Business", "Sci/Tech"]
max_tokens = 50
BATCH_SIZE = 256

In [34]:
def collate_batch(batch):
    y, x = list(zip(*batch))

    x = [vocab(tokeniser(text)) for text in x]
    x = [t + ([0]*(max_tokens - len(t))) if len(t) < max_tokens else t[:max_tokens] for t in x]

    return torch.tensor(x, dtype=torch.int32), torch.tensor(y, dtype=torch.int32) - 1

In [35]:
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, collate_fn=collate_batch, shuffle = True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, collate_fn=collate_batch, shuffle = True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, collate_fn=collate_batch, shuffle = True)

### Let us build our RNN model

In [36]:
EMBEDDING_SIZE = 300
NEURONS = 128
LAYERS = 2
NUM_CLASSES = 4

In [37]:
class RecurrentModel(nn.Module):
    def __init__(self, embed_size, hidden, layers, num_classes):
        super().__init__()
        self.embedding_layer = nn.Embedding(num_embeddings=len(vocab),
                                            embedding_dim=embed_size)

        self.rnn = nn.GRU(input_size=embed_size,
                          hidden_size=hidden,
                          num_layers=layers,
                          batch_first=True,
                          dropout=0.5)
        
        self.fc = nn.Linear(hidden, num_classes)

    def forward(self, x):
        embedded = self.embedding_layer(x)

        rnn_out, _ = self.rnn(embedded)

        final_hidden_state = rnn_out[:, -1, :]

        out = self.fc(final_hidden_state)
        return out

In [38]:
def accuracy(model, loader):
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for text, labels in loader:
            text, labels = text.to(device), labels.to(device)
            outputs = model(text)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return correct / total

In [ ]:
def train(model, optimiser, epochs=100):
    model.train()
    criterion = nn.CrossEntropyLoss()

    for epoch in range(epochs):
        total_loss = 0
        for text, labels in train_loader:
            text, labels = text.to(device), labels.to(device)

            labels = labels.long()

            optimiser.zero_grad()

            outputs = model(text)
            loss = criterion(outputs, labels)

            loss.backward()
            optimiser.step()

            total_loss += loss.item()

        average_loss = total_loss / len(train_loader)
        print(f"Epoch {epoch+1}/{epochs}, Avereage loss: {average_loss:.4f}")


In [40]:
epochs = 10
lr = 0.001

rnn_model = RecurrentModel(EMBEDDING_SIZE, NEURONS, LAYERS, NUM_CLASSES)
optimiser = torch.optim.Adam(rnn_model.parameters(), lr=lr)

In [41]:
train(rnn_model, optimiser=optimiser,  epochs=epochs)

Epoch [1/10], Loss: 0.5958
Epoch [2/10], Loss: 0.2412
Epoch [3/10], Loss: 0.1678
Epoch [4/10], Loss: 0.1189
Epoch [5/10], Loss: 0.0844
Epoch [6/10], Loss: 0.0598
Epoch [7/10], Loss: 0.0437
Epoch [8/10], Loss: 0.0342
Epoch [9/10], Loss: 0.0276
Epoch [10/10], Loss: 0.0213


In [42]:
print(f'{accuracy(rnn_model, test_loader):.4f}')

0.9064


In [53]:
NUM_SAMPLES = 5

In [58]:
def sample_text(model, loader):
    model.eval()
    samples = []
    
    with torch.no_grad():
        for text, labels in loader:
            text, labels = text.to(device), labels.to(device)
            outputs = model(text)
            _, predicted = torch.max(outputs, 1)
            
            for i in range(NUM_SAMPLES):
                sample_text = " ".join([vocab.get_itos()[idx] for idx in text[i].tolist() if idx != 0])
                samples.append((sample_text, labels[i].item(), predicted[i].item()))
            
            break
    
    print("\nPredictions:")
    for text, actual, pred in samples:
        print(f"Text: {text}\nActual: {labels[actual]}\nPredicted: {labels[pred]}\n\n")


In [59]:
sample_text(rnn_model, test_loader)


Predictions:
Text: making it look easy cleveland -- their membership in the nfl elite entitles the patriots to a gimme from time to time , like yesterday ' s 42-15 shellacking of the hapless cleveland browns .
Actual: 2
Predicted: 2


Text: justices to debate mail-order wine being freelance wine critics may sound like a sweet gig , but ray and eleanor have soured on it . because their home state , michigan , blocks direct shipments from out-of-state
Actual: 1
Predicted: 0


Text: panathinaikos 2 , arsenal 2 arsenal keeper jens lehmann was left red-faced in athens as two costly mistakes ensured that a champions league victory slipped through his side #39 s fingers again .
Actual: 2
Predicted: 2


Text: iraq nuclear losses #39 a scandal #39 former un chief weapons inspector hans has said the loss of control of iraq #39 s nuclear sites by the us after it occupied the country was scandalous .
Actual: 1
Predicted: 1


Text: oil prices set a new record above \$50 singapore ( reuters ) - oil

In [ ]:
# create confusion matrix
pass

In [ ]:
plot_confusion_matrix(rnn_model, test_loader)
